In [1]:
### インポート ###
import numpy as np
import pandas as pd
import glob
from selenium.webdriver.chrome.service import Service
from datetime import datetime
import time
import re
import csv
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, TypeVar, Generic

In [2]:
# 設定ファイル読み込み
df_config_style = pd.read_excel("C:\\keiba\\tool\\config.xlsx", sheet_name="style", header=0)
# 脚質
STYLE_MAP = df_config_style.set_index('key')['value'].to_dict()
print(f'脚質: {STYLE_MAP}')
REVERSE_STYLE_MAP = {v: k for k, v in STYLE_MAP.items()}

脚質: {1: '逃げ', 2: '先行', 3: '差し', 4: '追込'}


In [5]:
def calculate_recent_time_index_avg(row, df):
    """
    特定の馬(horse_id)の直近2走の平均time_indexを計算する
    """
    target_id = row['horse_id']
    target_date = row['race_date']
    
    # 1. dfから同じhorse_id、かつ今回のrace_dateより前のデータを抽出
    # ※昇順にソート（直近が下に来るようにする）
    df_filtered = df[
        (df['horse_id'] == target_id) & 
        (df['race_date'] < target_date)
    ].sort_values('race_date')
    
    # 2. time_indexがNaNでないものを抽出
    df_valid = df_filtered.dropna(subset=['time_index'])
    
    # 3. 有効なデータが2つ以上あるか判定
    if len(df_valid) >= 2:
        # 下から2つ（直近2走）を選択して平均を出す
        avg_index = df_valid.tail(2)['time_index'].mean()
        return avg_index
    else:
        # 2回未満ならNaN
        return np.nan

def calculate_jockey_win_rate(row, df):
    """
    特定の騎手(jockey_id)の直近100走（有効データ）の勝率を計算する
    """
    target_id = row['jockey_id']
    target_date = row['race_date']
    
    # 1. その騎手の、今回のレース日より前のデータを抽出
    # 2. かつ finish_rank が NaN でないものに絞る
    # 3. 日付順（昇順）にソート
    df_jockey_past = df[
        (df['jockey_id'] == target_id) & 
        (df['race_date'] < target_date) &
        (df['finish_rank'].notna())
    ].sort_values('race_date')
    
    # 4. 直近100レコードを抽出（100個なければ全件）
    df_recent = df_jockey_past.tail(100)
    
    # レコード数をカウント（分母）
    total_count = len(df_recent)    

    if total_count == 0:
        return np.nan
    
    # 5. finish_rank が 1 であるレコード数をカウント（分子）
    win_count = (df_recent['finish_rank'].astype(int) == 1).sum()
    
    # 勝率を計算して返す
    return win_count / total_count

def predict_position_half_type(group):
    '''
    以下でソート
    1. 脚質順
    2. タイム指数順
    '''
    group = group.sort_values(["style_id", "time_index_avg_recent_2"], ascending=[True, False])
    n = len(group)
    mid = n // 2
    res = pd.Series(index=group.index, dtype=str)
    res.iloc[:mid] = "front"
    res.iloc[mid:] = "back"
    return res

In [7]:
### 学習データ読み込み ###
print("trainデータ読み込み：開始")
df_train = pd.read_csv("C:\\keiba\\tool\\train\\train.csv", header=0)
print("trainデータ読み込み：終了")
### shutubaデータ読み込み ###
print("shutubaデータ読み込み：開始")
shutuba_files = glob.glob("C:\\keiba\\tool\\shutuba\\shutuba*.csv")
if shutuba_files:
    df_shutuba = pd.read_csv(shutuba_files[0], header=0)
else:
    print("該当するファイルが見つかりませんでした。")
print("shutubaデータ読み込み：終了") 

trainデータ読み込み：開始
trainデータ読み込み：終了
shutubaデータ読み込み：開始
shutubaデータ読み込み：終了


In [23]:
# =============
# 説明変数作成
# =============
print("説明変数作成：開始")
df_pred = df_shutuba.copy()

### 予想タイム指数 ###
print("    予想タイム指数計算：開始")
# 直近過去2レースのタイム指数の平均値
df_train['time_index_avg_recent_2_race_avg'] = df_train.groupby('race_id')['time_index_avg_recent_2'].transform('mean')
df_train['time_index_pred_from_race_avg'] = df_train['time_index_avg_recent_2'] - df_train['time_index_avg_recent_2_race_avg']
df_pred['time_index_avg_recent_2'] = df_pred.apply(calculate_recent_time_index_avg, axis=1, args=(df_train,))
df_pred['time_index_avg_recent_2_race_avg'] = df_pred.groupby('race_id')['time_index_avg_recent_2'].transform('mean')
df_pred['time_index_pred_from_race_avg'] = df_pred['time_index_avg_recent_2'] - df_pred['time_index_avg_recent_2_race_avg']
# メモ：NaNをどう埋めるかは後で考える。front/back判定でも使う
print("    予想タイム指数計算：終了")

### 騎手勝率 ###
print("    騎手勝率計算：開始")
# df_pred['jockey_win_rate_recent_100'] = df_pred.apply(calculate_jockey_win_rate, axis=1, args=(df_train,))
print("    騎手勝率計算：開始")

### コース×頭数×馬番別勝率
print("    コース×頭数×馬番別勝率計算：開始")
keys = ['racecourse', 'ground', 'distance', 'direction', 'num_horses', 'horse_number']
stats_df = df_train.groupby(keys)['finish_rank'].agg([
    ('win_count', lambda x: (pd.to_numeric(x, errors='coerce') == 1).sum()),
    ('total_count', 'count')
]).reset_index()
stats_df['condition_win_rate'] = stats_df['win_count'] / stats_df['total_count']
df_pred = pd.merge(df_pred, stats_df[keys + ['condition_win_rate']], on=keys, how='left')
print("    コース×頭数×馬番別勝率計算：終了")

### ポジションに紐づく勝率計算 ###
# trainデータに対するfront / backの割り当て
print("    ポジション（front / back）予想作成：開始")
# 方法(1) style_predとtime_index_avg_recent_2_race_avgからfront / backを予想
df_train["style_id"] = df_train["style_pred"].map(REVERSE_STYLE_MAP)
df_train["position_half_type_pred"] = df_train.groupby("race_id", group_keys=False).apply(predict_position_half_type)
# 方法(2) trainデータ全体にわたり、各馬がfrontとbackどちらの傾向が強いかを平均値により判定
df_train['finish_rank_num'] = pd.to_numeric(df_train['finish_rank'], errors='coerce')
df_train['relative_rank'] = df_train['finish_rank_num'] / df_train['num_horses']
relative_rank_avg = df_train.groupby('horse_id')['relative_rank'].transform('mean')
df_train['position_half_type_attr'] = np.where(
    relative_rank_avg.isna(), 
    np.nan, 
    np.where(relative_rank_avg < 0.5, 'front', 'back')
)
# predデータにおける最終コーナーの通過位置（front / back）予想
df_pred["style_id"] = df_pred["style_pred"].map(REVERSE_STYLE_MAP)
df_pred["position_half_type_pred"] = df_pred.groupby("race_id", group_keys=False).apply(predict_position_half_type)
print("    ポジション（front / back）予想作成：終了")
# 予想ポジション別勝率
print("    予想ポジション別勝率計算：開始")
# trainデータから方法(1)(2)のいずれかによりコース×頭数×(front/back)ごとの勝率を計算
selected_position_half_type = 'position_half_type_pred' # 'position_half_type_pred'または'position_half_type_attr'
keys = ['racecourse', 'ground', 'distance', 'direction', 'num_horses', selected_position_half_type]
stats = df_train.groupby(keys)['finish_rank'].agg([
    ('win_count', lambda x: (pd.to_numeric(x, errors='coerce') == 1).sum()),
    ('total_count', 'count')
]).reset_index()
stats['position_half_type_win_rate'] = stats['win_count'] / stats['total_count']
# predデータのコース×頭数×(front/back)に紐づく勝率を割り当てる
df_pred = pd.merge(
    df_pred, 
    stats[keys + ['position_half_type_win_rate']], 
    on=keys, 
    how='left'
)
print("    予想ポジション別勝率計算：終了")
# 予想ポジション別比率勝率
df_train['front_count'] = df_train.groupby('race_id')[selected_position_half_type].transform(lambda x: (x == 'front').sum())
df_train['back_count'] = df_train.groupby('race_id')[selected_position_half_type].transform(lambda x: (x == 'back').sum())
keys = ['num_horses', 'front_count', 'back_count', selected_position_half_type]
stats = df_train.groupby(keys)['finish_rank'].agg(
    composition_pos_win_rate = lambda x: (pd.to_numeric(x, errors='coerce') == 1).sum() / len(x)
).reset_index()
df_pred['front_count'] = df_pred.groupby('race_id')[selected_position_half_type].transform(lambda x: (x == 'front').sum())
df_pred['back_count'] = df_pred.groupby('race_id')[selected_position_half_type].transform(lambda x: (x == 'back').sum())
df_pred = pd.merge(df_pred, stats, on=keys, how='left')
print("説明変数作成：終了")

説明変数作成：開始
    予想タイム指数計算：開始
    予想タイム指数計算：終了
    騎手勝率計算：開始
    騎手勝率計算：開始
    コース×頭数×馬番別勝率計算：開始
    コース×頭数×馬番別勝率計算：終了
    ポジション（front / back）予想作成：開始


C:\Users\ryo\AppData\Local\Temp\ipykernel_15120\3222937692.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train["position_half_type_pred"] = df_train.groupby("race_id", group_keys=False).apply(predict_position_half_type)
C:\Users\ryo\AppData\Local\Temp\ipykernel_15120\3222937692.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_pred["position_half_type_pred"] = df_pred.groupby("race_id", grou

    ポジション（front / back）予想作成：終了
    予想ポジション別勝率計算：開始
     racecourse ground  distance direction  num_horses  \
0            佐賀      ダ       900         右           3   
1            佐賀      ダ       900         右           3   
2            佐賀      ダ       900         右           4   
3            佐賀      ダ       900         右           4   
4            佐賀      ダ       900         右           5   
...         ...    ...       ...       ...         ...   
1685         高知      ダ      1900         右          11   
1686         高知      ダ      1900         右          12   
1687         高知      ダ      1900         右          12   
1688         高知      ダ      2400         右          12   
1689         高知      ダ      2400         右          12   

     position_half_type_pred  win_count  total_count  \
0                       back          0            0   
1                      front          1            3   
2                       back          2            5   
3                      front

PermissionError: [Errno 13] Permission denied: 'C:\\keiba\\tool\\test\\test.csv'

In [27]:
# =====
# 学習
# =====


df_pred.to_csv("C:\\keiba\\tool\\pred\\pred.csv", encoding="cp932")


# =====
# 予想
# =====



# ===============
# シミュレーション
# ===============



